# AWS Textract Key Value Pairs

This example helps in identifying Key value pairs from multi-page PDFs using the Asynchronous API StartDocumentAnalysis. 


In [ ]:
# We would need to install the TRP module
!pip install amazon-textract-response-parser
import trp


In [ ]:
import boto3 

# Amazon Textract client
textract = boto3.client('textract')

# Call Amazon Textract

s3_bucket_name = "s3-read-test"
document_name = "OptusBill23Feb2023.pdf"

text_analysis =textract.start_document_analysis(
    DocumentLocation=
    {'S3Object': {
        'Bucket': s3_bucket_name, 
        'Name': document_name}},
    FeatureTypes=['FORMS', 'TABLES']
) 

job_id = text_analysis["JobId"]
job_id 

In [ ]:
job_response = textract.get_document_analysis(
        JobId=job_id
    )

nextToken = None
if('NextToken' in job_response):
    nextToken = job_response['NextToken']
while(nextToken):
    response = textract.get_document_analysis(JobId=job_id, NextToken=nextToken)
    job_response['Blocks'].extend(response['Blocks'])
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']
        
doc_response = trp.Document(job_response)
    

In [ ]:
detectedText = {}

for page in doc_response.pages:
    for field in page.form.fields:
        if field.value:
            detectedText[field.key.text] = field.value.text
        else:
            detectedText[field.key.text] = ""

detectedText